In [1]:
import pandas as pd

In [2]:
import mlflow

In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///C:/Users/USER/Desktop/MLFlow_Spam/mlruns/276831911855750534', creation_time=1693636723439, experiment_id='276831911855750534', last_update_time=1693636723439, lifecycle_stage='active', name='lemma_bow_v4', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/USER/Desktop/MLFlow_Spam/mlruns/714334430258642456', creation_time=1693636532712, experiment_id='714334430258642456', last_update_time=1693636532712, lifecycle_stage='active', name='lemmatoken_v4', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/USER/Desktop/MLFlow_Spam/mlruns/607675071012601167', creation_time=1693636284506, experiment_id='607675071012601167', last_update_time=1693636284506, lifecycle_stage='active', name='StemToken_v3', tags={}>,
 <Experiment: artifact_location='file:///C:/Users/USER/Desktop/MLFlow_Spam/mlruns/921301241704540182', creation_time=1693631832605, experiment_id='921301241704540182', last_update_time=1693631832605, lifecycle_stage='active', na

In [4]:
for exp in mlflow.search_experiments():
    print(exp.name, exp.experiment_id)

lemma_bow_v4 276831911855750534
lemmatoken_v4 714334430258642456
StemToken_v3 607675071012601167
Spam_V2 921301241704540182
Default 0


In [5]:
for i in exp:
    print(i)

('artifact_location', 'mlflow-artifacts:/0')
('creation_time', 1693631360427)
('experiment_id', '0')
('last_update_time', 1693631360427)
('lifecycle_stage', 'active')
('name', 'Default')
('tags', {})


In [8]:

Run_ID = '8e8f62a1f59c4a209da509a78c6efb38'

# load from run id
logged_vect = f'runs:/{Run_ID}/vectorizer'
logged_model = f'runs:/{Run_ID}/model'
loaded_vect = mlflow.sklearn.load_model(logged_vect)
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [9]:
run_id = str(loaded_model).split()[-1]
logged_vect = f'runs:/{run_id}/vectorizer'
loaded_vect = mlflow.sklearn.load_model(logged_vect)

In [10]:
loaded_vect

TfidfVectorizer()

In [11]:
import streamlit as st  
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(text.lower())
    # filtered_words = [word for word in words if word.isalpha()]
    # filtered_words = [word for word in words if word.isalpha() and word not in stop_words]
    # filtered_words = [stemmer.stem(word) for word in words if word.isalpha() and word not in stop_words]
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(filtered_words)

In [12]:
text = "URGENT! You have won a 1 week FREE membership in our Â£100,000 Prize Jackpot!"

# get the run_id
Run_ID = '8e8f62a1f59c4a209da509a78c6efb38'

# load from run id
logged_vect = f'runs:/{Run_ID}/vectorizer'
logged_model = f'runs:/{Run_ID}/model'
loaded_vect = mlflow.sklearn.load_model(logged_vect)
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Preprocess the new raw text
processed_text = preprocess_text(text)

# Vectorize the processed text
vectorized_text = loaded_vect.transform([processed_text])

# Make prediction
prediction = loaded_model.predict(vectorized_text)

# Print the prediction
print(f"Predicted label: {prediction[0]}")

Predicted label: spam


In [15]:
# This will fetch all registered models   
reg_models = mlflow.search_model_versions()

len(reg_models)

2

In [16]:
mlflow.search_model_versions()

[<ModelVersion: aliases=[], creation_timestamp=1693638108904, current_stage='Production', description='', last_updated_timestamp=1693638414043, name='spamfilter', run_id='8e8f62a1f59c4a209da509a78c6efb38', run_link='', source='file:///C:/Users/USER/Desktop/MLFlow_Spam/mlruns/714334430258642456/8e8f62a1f59c4a209da509a78c6efb38/artifacts/vectorizer', status='READY', status_message=None, tags={}, user_id=None, version=1>,
 <ModelVersion: aliases=[], creation_timestamp=1693638141555, current_stage='None', description='', last_updated_timestamp=1693638141555, name='spamfilter', run_id='cca7670e199e4358acb0b9324b8f6962', run_link='', source='file:///C:/Users/USER/Desktop/MLFlow_Spam/mlruns/276831911855750534/cca7670e199e4358acb0b9324b8f6962/artifacts/vectorizer', status='READY', status_message=None, tags={}, user_id=None, version=2>]

In [17]:

reg_models[0]

<ModelVersion: aliases=[], creation_timestamp=1693638108904, current_stage='Production', description='', last_updated_timestamp=1693638414043, name='spamfilter', run_id='8e8f62a1f59c4a209da509a78c6efb38', run_link='', source='file:///C:/Users/USER/Desktop/MLFlow_Spam/mlruns/714334430258642456/8e8f62a1f59c4a209da509a78c6efb38/artifacts/vectorizer', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [18]:
[[model.name, model.version, model.status, model.current_stage] for model in mlflow.search_model_versions()]

[['spamfilter', 1, 'READY', 'Production'], ['spamfilter', 2, 'READY', 'None']]

In [19]:
def get_production_model():
    try:        
        # This will fetch all registered models    
        prod = [model for model in mlflow.search_model_versions() if model.name == 'spamfilter' and model.current_stage == 'Production'] 
        model = prod[0]            
        print(f"Run_ID: {model.run_id}")
        print(f"Model Name: {model.name}")
        print(f"Version: {model.version}")               
    except:
        print("No Production Model Found")

get_production_model()

Run_ID: 8e8f62a1f59c4a209da509a78c6efb38
Model Name: spamfilter
Version: 1
